# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [38]:
import pandas as pd
import numpy as np
from scipy import stats

In [39]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [40]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [41]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [42]:
data.columns

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')

Question 1:  The ztest is appropriate because our sample size is 157
The CLT thoery applies because our sample is large enough such that the the mean of the sample should
be equal to the mean of the population

Question 2: The Null hypothesis is that the mean callback rate of 'black sounding' names should be equal to the 
callback rate of the population overall.  Alternate hypothesis is that some other factor (such 
as descrimination) is causing different callback rates

In [43]:
black_names = data[data['race'] == 'b']
white_names = data[data['race'] == 'w']
black_callback_rate = black_names.call.mean()
white_callback_rate = white_names.call.mean()
print('Black callback rate:', black_callback_rate, '   White callback rate:', white_callback_rate)

Black callback rate: 0.0644763857126236    White callback rate: 0.09650924056768417


In [44]:
# Clearly a big differnce.
# Let's use confidence interval of 95%, z=1.96

In [45]:
import math
std_error = math.sqrt((black_callback_rate*(1-black_callback_rate))/len(black_names) + (white_callback_rate*(1-white_callback_rate))/len(white_names))

In [46]:
zvalue = (white_callback_rate - black_callback_rate) / std_error
print('Zvalue = ', zvalue)
MOE = zvalue*std_error
print('Margin of error:', MOE)


Zvalue =  4.115550519002299
Margin of error: 0.03203285485506058


In [47]:
from statsmodels.stats.proportion import proportions_ztest
num_black = len(black_names[black_names['call'] == 1])
num_white = len(white_names[white_names['call'] == 1])
count = np.array([num_black, num_white])
nobs = np.array([len(black_names), len(white_names)])
stat, pval = proportions_ztest(count, nobs)
print('zvalue:', stat)
print('pvalue:', pval)

zvalue: -4.10841215243
pvalue: 3.98388683759e-05


Pvalue is tiny allowing us to reject the null hypothesis

When analyzing the callback rate for employment interviews of black sounding names and white sounding names
it was found that black sounding names received a statistically significant lower rate of callbacks.

This analysis does not show that black sounding names is the most relevant factor in determining callback
ratios.  To determine that we would ideally generate linear regressions and r-squared scores for various
items and determine which are correlated most strongly with low callback rates.